<h1>Weather Effects on NFL Play-By-Play Performance</h1>

A common thought as it pertains to the affects of weather on NFL games is that weather plays an important factor on how teams choose play calling and how they perform given certain weather conditions. We are aiming to understand how weather changes intragame affect how teams perform in the moment and ultimately help determine the outcome.

<h2>Background on American Football</h2> <br>
American Football is played between two teams in which each team is looking to advance a football down a 100 yard field to score a Touchdown (7 pts) or Field Goal (3 pts). Both teams are on the field at the same time, and the team advancing the football is called the <b>Offense (blue)</b> and the team trying to keep the Offense from advancing the football is call the <b>Defense (red).</b>

![Football_field](footballfield.jpg)

<h2>The Game</h2><br>
Each game is made up of 4 15 minute quarters. After the first two quarters, we have Halftime, where the teams will take a short break. Teams will flip a coin to see who starts with the ball at the beginning of the game, and the team that does not start with the ball at the beginning of the game will get the ball at halftime.<br>

<h3>A Series</h3><br>
The team on offense will four plays, or downs, to get 10 yards and reset their down counter. If a team does not get 10 yards on 4 downs, or gives the ball to the other defending team (turnover), then that team turns the ball over. A series ends when a team has turned the ball over or gotten ten or more yards.<br>

<h3>A Drive</h3><br>
A Drive is a collection of sequential series run by the offense until they have either score or turned the ball over (which also happens at halftime).


<h3>Playing conditions</h3>
Teams play in stadiums seating between 60 and 80k people. They range in design, but most fit into two categories; Open Air (or exposed to outside weather) or Enclosed (not exposed to outside weather)<br>

<b>The field can be made up of a couple types of surfaces:</b><br>
-Natural Grass<br>
-Artifical Turf (Used mostly for Enclosed Stadiums, but also in some Open Air Stadiums)<br>

<h2>The NFL</h2><br>
The NFL, or National Football League, is the American league for American Football, and considered the most prevalant in the world. There are 32 teams from across the continental USA split into two conferences call the AFC and NFC.

<img src="NFL_Logo.png" style=width:250px;height:350px>

<h2>The Teams</h2>

<img src="nfl_team_logos.png" style=width:650px;height:1050px>